# 改善精準度


## 檢查產出報表時 應該可以發現 非常多 「誤判」的情況


## 舉例

### 1. 註解 （ 不需要盤點註解 或 已被 Sunset 的 Code ）

```sql

-- 2.取得今日有變價資料 VIEW_DIM_TO_DW_ITEM_CODE JOIN  ERP_ITEMCODE_PRICE
DROP TABLE ${TEMPDB}.TP2_RF_IMEI_PRICE;
CREATE TABLE ${TEMPDB}.TP2_RF_IMEI_PRICE AS 
(
SELECT A.IMEI_TAC,B.ITEM_CODE
 ,B.ITEM_PRICE,B.UPDATE_DATE,B.TX_DATE 
FROM ${STAGEDB}.VIEW_DIM_TO_DW_ITEM_CODE A
JOIN ${STAGEDB}.ERP_ITEMCODE_PRICE B ON A.ERP_ITEM_CODE=B.ITEM_CODE
)WITH DATA PRIMARY INDEX(IMEI_TAC,ITEM_CODE);


```


### 2. 不講武德 任意換行魔人 （盤點不到 Table）

```sql

SELECT *
   FROM ${TEMPDB}.OCM_KPI_RENEW_MT_TMP5 A
   LEFT JOIN FOREIGN TABLE
        ( SELECT *
            FROM PDATA_VIEW.SUBSCR_PROM
        )@TD6750 B
     ON A.SUBSCR_ID = B.SUBSCR_ID
    AND A.PROM_SEQ_NO_DESC < B.PROM_SEQ_NO_DESC
    AND CAST(A.ORDER_COMPLETED_DTM AS DATE) > B.START_DATE
   LEFT JOIN 
        ${VIEWDB}.PROM_GRP_MT_VW C 
     ON B.PROM_ID = C.PROM_ID
  WHERE 1 = 1 
    AND A.ORDER_TYPE = 'CHCONTRACT' 

```


### 3. 特殊 Function （誤判 FROM）

```sql
SELECT   A.*
        ,EXTRACT(DAY FROM DIFF_DDHHMM) * 24 * 60 
        + EXTRACT(HOUR FROM DIFF_DDHHMM) * 60
        + EXTRACT(MINUTE FROM DIFF_DDHHMM) AS DIFF_MINS
FROM ${TEMPDB}.APP A

```




### 4. *字串問題 （這個需要各路大神來拯救）*

```sql
SELECT 
    head.*
    , 'ORIG_SUBSCR_ID = SUBSCR_ID to join SUBSCR_PROM'    AS PRE_CONTR_JOIN_TYPE          

FROM ${TEMPDB}.OCM_KPI_RENEW_MT_TMP5 head

```




## 如何改進誤判狀況



### *1. 主要邏輯判斷更精確* 


### *2. 懶得改主要邏輯 正規化 SQL Script* ✅






# 正規化


## 一樣使用 python re


## 讀進 SQL Script 時 就先正規化

In [ ]:
import pathlib
import re
import pandas as pd
import time

from IPython.display import clear_output, display


from itertools import zip_longest

# Remove Pandas DataFrame Default Truncate Value
pd.set_option('display.max_colwidth', None)

In [ ]:

# Script 檔案所在資料夾
path = 'Script/regex'

# 要抓取的檔案 全部都是 .txt 為副檔名
# File Pattern 
file_pattern = '*.txt'

p = pathlib.Path(path)

p.exists()

In [ ]:


# 以 副檔名皆為 .txt 來抓取
files = p.glob(file_pattern)

files = [f for f in files]

for f in files:
    print(f)

# 三個檔案中


### 01.txt  ☞  有著大量 「註解」 與 「Sunset Code」 問題


### 02.txt  ☞  有著 「特殊 Function 」 問題


### 03.txt  ☞  有著 「亂換行」 的問題 與 「字串問題」

<br>
# *邏輯與想法*


##  1. 註解可以全部清除 

- 單行中註解 有 

```sql 
-- 這是註解
SELECT * FROM A

```

- 多行  

```sql
/* 多行
        ABC  */
SELECT * FROM B
        
```


### 找到 註解的 regex pattern 一樣可以去 regex101.com 

[多行註解的解答 REGEX101.com](https://regex101.com/r/MzKW0d/1)

[單行註解的解答 ](https://regex101.com/r/hnDGJX/1)

```python

# 根據上面的解答 來填入
sql = re.sub(r'\/\*.*?\*\/', '', sql, flags=re.DOTALL|re.MULTILINE)

# 注意要把 Group 1 放回來
sql = re.sub(r'--.*\n*|(.*?)--.*\n*', r'\g<1>\n', sql)


```



<br><br>
## 2. 特殊 Function （針對 FROM 的 Function 就把 FROM 暫時移除）


### 找出特殊 function 的 關鍵字 *（這邊只找針對 FROM 且為 TD Function ）*

- TRIM

- EXTRACT

- SUBSTRING


### 接著找 regex pattern 

[以 EXTRACT 為例](https://regex101.com/r/mDF0aD/1)


### 接著就按圖施工 ~~不保證成功~~

```python

# 我們只拿掉 from 剩下的仍然要 故取 group1 group2
sql = re.sub(r'(\bEXTRACT\b *\(.*?)\bfrom\b(.*?\))', r'\g<1> \g<2>', sql, flags=re.I)


```


### *其實 三個都類似 我們只找一種 通用 pattern 再置換關鍵字 等等實際做*



<br><br>
## 3. 亂換行 就把換行符號去掉

### 主邏輯是找出 SQL KeyWord 故把 KeyWord 旁邊的換行符號去掉

[以 JOIN 為例 ](https://regex101.com/r/1o0rMH/1)



```python


# 換行符號在 group 1 之外
sql = re.sub(r'(\bJOIN\b *)\n+', r'\g<1>', sql, flags=re.I)


```

### 可以順便試試看 把關鍵字換成 FROM



<br>


# 接下來就真的是按圖施工啦

## 可以先簡單測試




In [ ]:

# 第一個
try:
    with open(files[0]) as f:

        sql = f.read()
except UnicodeDecodeError:
    with open(files[0], encoding='ms950') as f:
        sql = f.read()

print('========== 修改前的 File============')
print(sql)

time.sleep(5)

clear_output()

sql = re.sub(r'\/\*.*?\*\/', '', sql, flags=re.DOTALL|re.MULTILINE)

sql = re.sub(r'--.*\n*|(.*?)--.*\n*', r'\g<1>\n', sql)

print('========== 修改後的 File============')

print(sql)






In [ ]:

# 第二個

try:
    with open(files[1]) as f:

        sql = f.read()
except UnicodeDecodeError:
    with open(files[1], encoding='ms950') as f:
        sql = f.read()

print('========== 修改前的 File============')

print(sql)

time.sleep(5)
clear_output()

sql = re.sub(r'(\bEXTRACT\b *\(.*?)\bfrom\b(.*?\))', r'\g<1> \g<2>', sql, flags=re.I)

print(sql)


In [ ]:

# 第三個
try:
    with open(files[2]) as f:

        sql = f.read()
except UnicodeDecodeError:
    with open(files[2], encoding='ms950') as f:
        sql = f.read()

print('========== 修改前的 File============')

print(sql)

time.sleep(10)
clear_output()

sql = re.sub(r'(\bJOIN\b *)\n+', r'\g<1>', sql, flags=re.I)

print(sql)


# 整合

## 額外事項

1. 寫一個備份 File 作為正規化後檔案

2. 針對第二點跟第三點的 KeyWord 延伸


In [ ]:


def clean_sql(file):
    
    # 讀檔
    try:
        with open(file, encoding='utf-8') as f:

            sql = f.read()
    except UnicodeDecodeError:
        with open(file, encoding='ms950') as f:
            sql = f.read()
            
    # clean /* */ 註解
    sql = re.sub(r'\/\*.*?\*\/', '', sql, flags=re.DOTALL|re.MULTILINE)
    
    # clean -- 註解
    sql = re.sub(r'--.*\n*|(.*?)--.*\n*', r'\g<1>\n', sql)
        
    # 第二點的 keyword 延伸
    sp_function_keyword = [
        'TRIM',
        'EXTRACT',
        'SUBSTRING',
    ]
    # 關鍵字的地方 給他一個 {} 來讓 字串填入
    sp_function_pattern = r'(\b{}\b *\(.*?)\bfrom\b(.*?\))'
    # List Comprehension
    regex_sp_function = [re.compile(sp_function_pattern.format(kw), flags=re.I) for kw in sp_function_keyword]
    
    for regex in regex_sp_function:
        # 不需要 flags 了 前面已經 compile 過 且加入 flags 了
        sql = re.sub(regex, r'\g<1> \g<2>', sql)
        
    # 第三點的 keyword 延伸
    sql_kw = [
    'UPDATE',
    'DELETE',
    'FROM',
    'REPLACE VIEW',
    'JOIN',
    'INSERT INTO',
    'DROP',
    'DROP TABLE',
    'CREATE', 
    ]
    sql_kw_pattern = r'(\b{}\b *)\n+'
    regex_sql = [re.compile(sql_kw_pattern.format(kw), flags=re.I) for kw in sql_kw]
    
    for regex in regex_sql:
        sql = re.sub(regex, r'\g<1>', sql)
        
    # 額外寫檔出來 我們都寫進 子資料夾 bk b_打頭
    b_file = file.parent / 'bk' / ('b_' + file.name)
    
    if not b_file.parent.exists():
        b_file.parent.mkdir()
    
    with open(b_file, 'w', encoding='ms950') as f:
        f.write(sql)
        
    return sql.split('\n')
    
    
    



In [ ]:
clean_sql(files[0])

# 字串問題


## 這一題需要各路大神想想了


### 如果直接捨棄 那麼下面這一種就完全沒辦法盤點

```sql

select 'select * from PDATA.TABLENAME ' ||
       'where txoffset = '' ' || job.timewindowend || ' '' '
from CONTROLTABLE


```